In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
austin = pd.read_csv("../data/Austin housing market data/austin housing clean.csv")
austin.drop(columns=['Unnamed: 0'], inplace = True)
austin.head()

In [ ]:
tx_school = pd.read_csv('../data/Austin housing market data/Texas_zip_school_median.csv')
tx_school.head()

In [ ]:
temp = austin.merge(tx_school, how = 'left', left_on='zip_code', right_on='Postal')
temp.rename(columns={"Score": "school_rating"}, inplace=True)
#len(austin)

In [ ]:
mortgage = pd.read_csv("../data/Data_mortgage_stocks/mortgage_for_austin.csv")
mortgage.head()

In [ ]:
temp2 = temp.merge(mortgage, how = 'left', left_on="period_begin", right_on = "Year_Month")
temp2.drop(columns=["Postal", "Year_Month"], inplace=True)
temp2.head()

In [ ]:
DGL = pd.read_csv("../data/Data_mortgage_stocks/stock_DGL_for_austin.csv")
temp3 = temp2.merge(DGL, how = "left", left_on = "period_begin", right_on = "Date_3Mo")
temp3.drop(columns = ["Date_3Mo"], inplace = True)
temp3.rename(columns = {"Close": "DGL"}, inplace=True)
temp3.head()


In [ ]:
QQQ = pd.read_csv("../data/Data_mortgage_stocks/stock_QQQ_for_austin.csv")
QQQ.head()
temp4 = temp3.merge(QQQ, how = "left", left_on = "period_begin", right_on = "Date_3Mo")
temp4.drop(columns = ["Date_3Mo"], inplace = True)
temp4.rename(columns = {"Close": "QQQ"}, inplace=True)
temp4.head()

In [ ]:
SPY = pd.read_csv("../data/Data_mortgage_stocks/stock_SPY_for_austin.csv")
SPY.head()
temp5 = temp4.merge(SPY, how = "left", left_on = "period_begin", right_on = "Date_3Mo")
temp5.drop(columns = ["Date_3Mo"], inplace = True)
temp5.rename(columns = {"Close": "SPY"}, inplace=True)
temp5.head()

In [ ]:
inflation = pd.read_csv("../data/InflationRate/US_inflation_rate.csv")
inflation['Date'] = pd.to_datetime(inflation['Date'])
inflation = inflation.sort_values(['Date']).reset_index(drop = True)
#inflation = inflation.groupby(['Date'])
inflation = inflation.groupby(inflation['Date'].dt.to_period('Q'))['Inflation_Rate'].median().to_frame()
inflation.reset_index(inplace=True)

inflation = inflation.astype({"Date": "str"})

inflation['Date'] = pd.to_datetime(
    inflation['Date'].str.replace(r'(\d+)(Q\d)', r'\1\2'), errors='coerce')
inflation = inflation.astype({"Date": "str"})
inflation.head()

In [ ]:
temp6 = temp5.merge(inflation, how = 'left',  left_on = "period_begin", right_on = "Date")
temp6.drop(columns = ["Date"],inplace=True)
temp6.head()
del temp5, temp4, temp3, temp2, temp

In [ ]:
median_income = pd.read_csv("../data/median_income/median_income_TX_clean.csv")
temp7 = temp6.merge(median_income, how = "left", left_on = ["zip_code", "Year"], right_on = ["zipcode", "year"])
temp7.drop(columns = ['zipcode', 'year'],inplace = True)
temp7.head()

In [ ]:
unemploy_temp = pd.read_csv("../data/unemployment_rate/unemployment_austin6counties_clean.csv")
unemploy = pd.melt(unemploy_temp, id_vars = ['Zip Code'], value_vars = [str(i) for i in range(2011,2021)])
unemploy = unemploy.astype({"variable": "int64"})
temp8 = temp7.merge(unemploy, how = "left", left_on= ["zip_code", "Year"], right_on= ["Zip Code", "variable"])
del temp7
temp8.drop(columns=["Zip Code", "variable"], inplace = True)
temp8.rename(columns={"value": "Unemployment"}, inplace = True)
temp8.head()

In [ ]:
del unemploy_temp, unemploy
mobility_abroad_temp = pd.read_csv("../data/mobility/mobility_abroad_austin_clean.csv")
mobility_withincounty_temp = pd.read_csv("../data/mobility/mobility_withincounty_austin_clean.csv")
mobility_withinstate_temp = pd.read_csv("../data/mobility/mobility_withinstate_austin_clean.csv")
mobility_withUS_temp = pd.read_csv("../data/mobility/mobility_withinUS_austin_clean.csv")


In [ ]:
mobility_abroad = pd.melt(mobility_abroad_temp,id_vars = ['Zip Code'], value_vars = [str(i) for i in range(2011,2021)])
mobility_abroad = mobility_abroad.astype({"variable": "int64"})

mobility_withincounty = pd.melt(mobility_withincounty_temp,id_vars = ['Zip Code'], value_vars = [str(i) for i in range(2011,2021)])
mobility_withincounty = mobility_withincounty.astype({"variable": "int64"})

mobility_withinstate = pd.melt(mobility_withinstate_temp,id_vars = ['Zip Code'], value_vars = [str(i) for i in range(2011,2021)])
mobility_withinstate = mobility_withinstate.astype({"variable": "int64"})

mobility_withUS = pd.melt(mobility_withUS_temp,id_vars = ['Zip Code'], value_vars = [str(i) for i in range(2011,2021)])
mobility_withUS = mobility_withUS.astype({"variable": "int64"})

temp9 = temp8.merge(mobility_abroad, how = "left", left_on= ["zip_code", "Year"], right_on= ["Zip Code", "variable"])
temp9.drop(columns=["Zip Code", "variable"], inplace = True)
temp9.rename(columns={"value": "mobility_abroad"}, inplace = True)

temp10 = temp9.merge(mobility_withincounty, how = "left", left_on= ["zip_code", "Year"], right_on= ["Zip Code", "variable"])
temp10.drop(columns=["Zip Code", "variable"], inplace = True)
temp10.rename(columns={"value": "mobility_withincounty"}, inplace = True)

temp11 = temp10.merge(mobility_withinstate, how = "left", left_on= ["zip_code", "Year"], right_on= ["Zip Code", "variable"])
temp11.drop(columns=["Zip Code", "variable"], inplace = True)
temp11.rename(columns={"value": "mobility_withinstate"}, inplace = True)

temp12 = temp11.merge(mobility_withUS, how = "left", left_on= ["zip_code", "Year"], right_on= ["Zip Code", "variable"])
temp12.drop(columns=["Zip Code", "variable"], inplace = True)
temp12.rename(columns={"value": "mobility_withUS"}, inplace = True)
temp12.to_csv("../data/Austin housing market data/clean_austin_price_reg_train.csv")

